In [2]:
import os
import numpy as np

In [3]:
path_home = os.path.expanduser('~/')
path_67p = path_home+'Documents/year1/shape_modelling/67p/'

In [4]:
# Load in apparent magnitudes (determined from phase func and shape model)
mag_file = path_67p+'67P_LCs_R.dat'
f = open(mag_file, 'r')
lines = f.readlines()
r_mags = []
for line in lines[1:]:
    data = line.split()
    r_mags.append(float(data[2]))
# r_mags contains all the r-band photometry
# if the LSST filter is r, we make no conversion
# for filters ugizy, will need to use (solar) colour indices to convert

In [5]:
# Read in output from SQL 5sigmaDepth query
# If you reset this, go and remove parenthesis manually
sql_file = path_67p+'LSST_SQL_output.txt'
f = open(sql_file, 'r')
lines = f.readlines()
filt_list = []; five_sig_depth = []
for line in lines:
    data = line.split()
    filt_list.append(str(data[1]))
    five_sig_depth.append(float(data[2]))

In [6]:
# Now want to determine uncertainties using five sigma depths in every colour filter
# solar-analog colour indices to use:
u = -25.3
g = -26.52
r = -26.93
i = -27.05
z = -27.07
y = -27.07

# Define colour indices (x-r)
col_inds = [u-r,g-r,i-r,z-r,y-r]
col_inds

[1.629999999999999,
 0.41000000000000014,
 -0.120000000000001,
 -0.14000000000000057,
 -0.14000000000000057]

In [7]:
# Function to determine SNR from 5sigdepth

def determine_snr(m_sig5, m2):
    delta_m = m_sig5 - m2
    snr_unknown = 5./(10.**(-delta_m/2.5))
    return snr_unknown

In [8]:
# Find whether LSST detects object in obsID
# Calculate SNR of detection (incl. below 5sig)
snr_list = []
for i, filt in enumerate(filt_list):
    if filt == 'r':
        mag = r_mags[i]  
    if filt == 'u':
        mag = r_mags[i]+col_inds[0]
    if filt == 'g':
        mag = r_mags[i]+col_inds[1]
    if filt == 'i':
        mag = r_mags[i]+col_inds[2]
    if filt == 'z':
        mag = r_mags[i]+col_inds[3]
    if filt == 'y':
        mag = r_mags[i]+col_inds[4]
    snr = determine_snr(five_sig_depth[i], mag)
    print('snr=%.4f, %smag =%.4f, 5-sig depth=%.4f'%(snr, filt, mag, five_sig_depth[i]))
    snr_list.append(snr)

snr=2.7827, ymag =22.3125, 5-sig depth=21.6762
snr=3.0829, ymag =22.0901, 5-sig depth=21.5651
snr=7.6688, zmag =22.2596, 5-sig depth=22.7240
snr=11.5521, zmag =22.0878, 5-sig depth=22.9970
snr=16.5478, imag =22.1612, 5-sig depth=23.4606
snr=13.6144, imag =22.1612, 5-sig depth=23.2488
snr=12.8458, zmag =22.0400, 5-sig depth=23.0645
snr=13.1648, zmag =22.0397, 5-sig depth=23.0908
snr=12.6936, zmag =22.0429, 5-sig depth=23.0544
snr=12.9730, zmag =22.0442, 5-sig depth=23.0794
snr=5.1134, ymag =22.1875, 5-sig depth=22.2119
snr=33.6683, rmag =22.2619, 5-sig depth=24.3325
snr=9.6339, ymag =21.8515, 5-sig depth=22.5636
snr=8.8930, ymag =21.8886, 5-sig depth=22.5138
snr=25.2347, imag =21.8941, 5-sig depth=23.6517
snr=5.6915, ymag =22.0649, 5-sig depth=22.2055
snr=8.7722, zmag =21.9777, 5-sig depth=22.5880
snr=8.6992, zmag =21.9694, 5-sig depth=22.5707
snr=9.7493, zmag =21.9534, 5-sig depth=22.6784
snr=9.3304, zmag =21.9523, 5-sig depth=22.6296
snr=8.8576, zmag =21.9965, 5-sig depth=22.6174
snr=

In [9]:
unc_list = 1./np.array(snr_list)
unc_list

array([0.35936202, 0.32436978, 0.13039873, 0.08656457, 0.06043096,
       0.0734518 , 0.07784674, 0.07596011, 0.07877993, 0.07708332,
       0.19556383, 0.02970152, 0.10380033, 0.11244821, 0.03962792,
       0.1757009 , 0.11399615, 0.11495263, 0.10257194, 0.10717673,
       0.11289695, 0.04561484, 0.08037037, 0.06551486, 0.02342806,
       0.03161515, 0.02870192, 0.02780595, 0.02368137, 0.02395513,
       0.02790004, 0.02807116, 0.03839997, 0.03801329, 0.06742609,
       0.06640443, 0.02438736, 0.03606692, 0.03112374, 0.03888732,
       0.16107054, 0.1134684 , 0.17327813, 0.08617921, 0.04439523,
       0.05448705, 0.05336205, 0.023329  , 0.02737716, 0.10225657,
       0.08929337, 0.08627834, 0.01776691, 0.01646773, 0.0144935 ,
       0.01428468, 0.01847064, 0.01332797, 0.09576167, 0.08023552,
       0.08712339, 0.09210745, 0.09393878, 0.06815867, 0.11145314,
       0.10849978, 0.10332387, 0.03636539, 0.02692089, 0.02328279,
       0.02400621, 0.03173453, 0.02563198, 0.02093143, 0.01014

In [10]:
fout = open(path_67p+'LSST_uncertainties_from_colours_ALL.txt', 'w+')
for val in unc_list:
    fout.write(str('%.4f'%val)+'\n')
fout.close()